# Exploración del dataset


In [29]:
import pandas as pd
import rdflib
import numpy

In [30]:
df = pd.read_csv('dataset/raw.csv.gz', dialect='excel', keep_default_na=False, dtype=str, index_col=0)

El dataset tiene un indice que no se corresponde con la cantidad de filas del dataset, sino con el dataset original. Para acceder a las filas se puede usar `.loc[indice]` en lugar de `.iloc[indice]`.

In [ ]:
df.index

In [ ]:
df.loc[764981]  # .loc hace referencia al indice

In [ ]:
df.iloc[764981] # .iloc hace referencia al número de fila dentro del dataset

Obtener mis las lineas en las que tengo que trabajar.

In [ ]:
df.iloc[0:200]

☝️ Chequear que tenga 200 filas

In [ ]:
Obtener los candidatos duplicados para una fila

In [ ]:
df.loc[199].dup_candidates

In [ ]:
Chequear las descripciones de una fila

In [ ]:
candidates = [int(x) for x in df.iloc[199].dup_candidates.split()] # Esto devuelve una lista de indices [990, 1200]
df.loc[candidates] # Esto devuelve las filas de un dataset en esa lista de indices

In [ ]:
# muestra las descripciones de cada aviso junto con sus indidces...
for idx, desc in zip(candidates,df.loc[candidates]['description']):
    print(f' {idx}   ******************')
    print(desc)
    print()

## El curado de los datos
Lo que debemos hacer es revisar los avisos a ver si los candidatos realmente están duplicados o no. -+En caso de no estar duplicados, la columna dup_candidates debe ser corregida. Suponga que estamos revisando la fila 199. El valor de la columna `dup_candidates` para la fila 199 es `'199 305206 624765'`, es decir que las filas 305206 y 624765 son duplicadas de la 199. No es el caso en este ejemplo, pero supongamos que la fila 305206 no hace referencia a la misma propiedad. Entonces, la versión corregida para `dup_candidates` debería contener solo `'199 624765'`. Por el contrario, si efectivamente todas las ocurrencias hablan de la misma propiedad, entonces la versión corregida para `dup_candidates` debería quedar exactamente igual, es decir `'199 305206 624765'`.

La idea es generar una nueva lista de 200 strings, con las columnas revisadas y agregarla a un nuevo dataset.

In [ ]:
# Inserte su metodología para detectar duplicados aquí!
# El resultado debe ser una lista de 200 strings.

revised = ['']*200
revised[0] = '0'
revised[1] = '1'
revised[2] = '2'
# ...
revised[199] = '199 305206 624765'; # <=== Cuidado con el orden de la lista!
revised[-10:] 

La nueva lista de strings puede tener valores en blanco (un string vacio `''`). Esto indica que esa fila no fue revisada. Además, el orden de los indices no importa, por lo tanto `'199 305206 624765'` es igual a `'199 624765 305206'` y a `'305206 624765 199'`. 

## Forma del entregable
El entregable será un nuevo dataset con una nueva columna llamada `dup_curated` el cual contrendrá un string nuevo por cada fila. Este string representará las filas que están duplicadas que han sido verificadas. Si las filas con indice 199, 305206 y 624765 están efectivamente duplicadas, como en el ejemplo anterior, la nueva columna debe quedar igual, es decir, para la fila con indice 199 tendrá el valor

`199 305206 624765`

Una vez que estemos contentos con la nueva lista revisada, vamos a crear un nuevo dataset y a agregar la nueva columna. Si el dataset no tiene la misma longitud que la lista, nos va a dar error! Ojo con eso y ojo con el orden de los elementos de la lista. 

In [31]:
def normalize_text(text: str) -> str:
    """Remove all symbols and whitespace, and normalize the text."""
    # Keep only alphanumeric characters and convert to lowercase
    return ''.join(char.lower() for char in text if char.isalnum())

def compare_texts(text1: str, text2: str) -> bool:
    """Compare two texts after normalization."""
    return normalize_text(text1) == normalize_text(text2)

def compare_entry(left: pd.Series, right: pd.Series) -> bool:
    """Compare two entries for address and description equality."""
    if not compare_texts(left['address'], right['address']):
        print('Address is different')
        return False
    
    if not compare_texts(left['description'], right['description']):
        print(f'Description is different -> {left["description"]} \n != \n{right["description"]}')
        return False
    
    return True

def verificar_duplicados(current_location: pd.Series) -> str:
    """Verify duplicates for the current entry based on dup_candidates."""
    indices = list(map(int, current_location['dup_candidates'].split()))
    
    # Initialize list to hold duplicate indices
    duplicate_indices = []

    # Compare current location with each of the other locations
    for index in indices:
        if index < 0 or index >= len(df):
            print(f'Index {index} is out of bounds, skipping.')
            continue
        
        if index != current_location.name and compare_entry(current_location, df.iloc[index]):
            duplicate_indices.append(str(index))

    # If duplicates were found, update dup_candidates
    if duplicate_indices:
        print(f'Found duplicates: {duplicate_indices}')
        return ' '.join(duplicate_indices)

    # Return original dup_candidates if no duplicates are found
    return current_location['dup_candidates']

In [32]:
START_INDEX=2400
END_INDEX=2600
newdf = df.iloc[START_INDEX:END_INDEX][['dup_candidates']]
newdf

,dup_candidates
2400,2202 2400 378078 384109
2401,1403 2345 2401 374538 384211
2402,2402 612575
2403,2403 349480 627095
2404,2404 324647 650873
...,...
2595,2595
2596,2596
2597,2597 376931
2598,2598 379887


In [33]:
newdf['dup_curated'] = df[START_INDEX:END_INDEX].apply(verificar_duplicados, axis=1)

Index 378078 is out of bounds, skipping.
Index 384109 is out of bounds, skipping.
Found duplicates: ['2202']
Description is different -> Departamento tres ambientes a estrenar, el mismo cuenta con living, comedor, cocina, baño completo, dos dormitorios con placard y balcon al frente
Detalles tecnicos:
Vidrios DVH (doble vidrio hermético) en ventanas balcón para los deptos que dan al frente.
      
    
Aberturas línea Mediterránea M-3, M-5 y Mónaco TOP 45 color grafito con manijas de apertura falleba.
Calefacción central mediante calderas y radiadores PEISA en todos los departamentos al frente. Nuestro innovador sistema permite mayor comodidad (libera espacios), seguridad, eficiencia energética y económica para los usuarios de calefacción. Cada departamento contara con termostato digital individual de regulación de temperatura y contador de horas.
Sistema central de agua caliente sanitaria, lo cual permite más espacio en cada cocina de tu departamento, espontaneidad en el calentamiento

In [34]:
newdf

,dup_candidates,dup_curated
2400,2202 2400 378078 384109,2202
2401,1403 2345 2401 374538 384211,1403
2402,2402 612575,2402 612575
2403,2403 349480 627095,2403 349480 627095
2404,2404 324647 650873,2404 324647 650873
...,...,...
2595,2595,2595
2596,2596,2596
2597,2597 376931,2597 376931
2598,2598 379887,2598 379887


In [35]:
grupo_nro = 13
newdf.to_csv(f'dataset/entrega_tis_grupo_{grupo_nro:02}.csv.gz', escapechar='"')